# RL Baselines3 Zoo: Training in Colab



Github Repo: [https://github.com/DLR-RM/rl-baselines3-zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

Stable-Baselines3 Repo: [https://github.com/DLR-RM/rl-baselines3-zoo](https://github.com/DLR-RM/stable-baselines3)


# Install Dependencies



In [ ]:
!apt-get install swig cmake ffmpeg freeglut3-dev xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


## Clone RL Baselines3 Zoo Repo

In [ ]:
!git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo

fatal: destination path 'rl-baselines3-zoo' already exists and is not an empty directory.


In [ ]:
%cd /content/rl-baselines3-zoo/

/content/rl-baselines3-zoo


### Install pip dependencies

In [ ]:
!pip install slimevolleygym
!pip install -r requirements.txt

## Train an RL Agent


The train agent can be found in the `logs/` folder.

Here we will train A2C on CartPole-v1 environment for 100 000 steps. 


To train it on Pong (Atari), you just have to pass `--env PongNoFrameskip-v4`

Note: You need to update `hyperparams/algo.yml` to support new environments. You can access it in the side panel of Google Colab. (see https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory)

In [ ]:
!python train.py --algo a2c --env SlimeVolley-v0 --n-timesteps 100000

Traceback (most recent call last):
  File "train.py", line 143, in <module>
    raise ValueError(f"{env_id} not found in gym registry, you maybe meant {closest_match}?")
ValueError: SlimeVolley-v0 not found in gym registry, you maybe meant 'no close match found...'?


#### Evaluate trained agent


You can remove the `--folder logs/` to evaluate pretrained agent.

In [ ]:
!python enjoy.py --algo a2c --env CartPole-v1 --no-render --n-timesteps 5000 --folder logs/

#### Tune Hyperparameters

We use [Optuna](https://optuna.org/) for optimizing the hyperparameters.

Tune the hyperparameters for PPO, using a tpe sampler and median pruner, 2 parallels jobs,
with a budget of 1000 trials and a maximum of 50000 steps

In [ ]:
!python train.py --algo ppo --env SlimeVolley-v0 --gym-packages slimevolleygym -n 20000000 -optimize --n-trials 10 --n-jobs 2 --sampler tpe --pruner median

========== SlimeVolley-v0 ==========
Seed: 3894817882
Default hyperparameters for environment (ones being tuned will be overridden):
OrderedDict([('ent_coef', 0.0),
             ('gae_lambda', 0.95),
             ('gamma', 0.99),
             ('n_envs', 128),
             ('n_epochs', 10),
             ('n_steps', 2048),
             ('n_timesteps', 20000000.0),
             ('normalize', True),
             ('policy', 'MlpPolicy')])
Using 128 environments
Overwriting n_timesteps with n=20000000
Doing 200 intermediate evaluations for pruning based on the number of timesteps. (1 evaluation every 100k timesteps)
Normalization activated: {'gamma': 0.99}
Optimizing hyperparameters
/usr/local/lib/python3.7/dist-packages/optuna/samplers/_tpe/sampler.py:266: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  ExperimentalWarning,
Sampler: tpe - Pruner: median
[I 2022-05-02 17:18:01,638] A new study created in memory with name: no-

### Record  a Video

In [ ]:
# Set up display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
!python -m utils.record_video --algo a2c --env CartPole-v1 --exp-id 0 -f logs/ -n 1000

### Display the video

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
show_videos(video_path='logs/a2c/CartPole-v1_1/videos/', prefix='')

### Continue Training

Here, we will continue training of the previous model

In [ ]:
!python train.py --algo a2c --env CartPole-v1 --n-timesteps 50000 -i logs/a2c/CartPole-v1_1/CartPole-v1.zip

In [ ]:
!python enjoy.py --algo a2c --env CartPole-v1 --no-render --n-timesteps 1000 --folder logs/